In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from sklearn.preprocessing import normalize
import time
import sys
from itertools import product

def load_data(file_path):
    """Load data from file and create sparse matrix."""
    print("Loading data...")
    rows, cols, data = [], [], []
    user_history = {}
    max_user_id, max_item_id = 0, 0
    
    with open(file_path, 'r') as f:
        for line in f:
            parts = list(map(int, line.strip().split()))
            if not parts:
                continue
            user_id = parts[0]
            items = parts[1:]
            if not items:
                continue
                
            user_history[user_id] = items
            max_user_id = max(max_user_id, user_id)
            for item_id in items:
                rows.append(user_id)
                cols.append(item_id)
                data.append(1)
                max_item_id = max(max_item_id, item_id)
                
    X = csr_matrix((data, (rows, cols)), shape=(max_user_id + 1, max_item_id + 1))
    print(f"Data loaded. Shape: {X.shape}, Non-zeros: {X.nnz}")
    return X, user_history

def create_train_val_split(user_history, X_shape):
    """Split data into train and validation sets."""
    train_hist, val_hist = {}, {}
    
    for user, items in user_history.items():
        if len(items) < 2:
            train_hist[user] = items
            val_hist[user] = []
        else:
            train_hist[user] = items[:-1]
            val_hist[user] = [items[-1]]
    
    rows, cols, data = [], [], []
    for user, items in train_hist.items():
        for item in items:
            rows.append(user)
            cols.append(item)
            data.append(1)
    
    X_train = csr_matrix((data, (rows, cols)), shape=X_shape)
    return X_train, train_hist, val_hist

def compute_item_similarity(X, k=50, normalization='l2', min_similarity=0.0):
    """
    Compute item-item similarity matrix.
    
    Args:
        X: User-item interaction matrix (sparse)
        k: Number of top neighbors to keep per item (0 = keep all)
        normalization: 'l1', 'l2', or None
        min_similarity: Minimum similarity threshold
    
    Returns:
        Sparse similarity matrix (item x item)
    """
    print(f"Computing ItemKNN similarity (k={k}, norm={normalization}, min_sim={min_similarity})...")
    start_time = time.time()
    
    # Normalize columns (items) for cosine similarity
    if normalization:
        X_norm = normalize(X, norm=normalization, axis=0)
    else:
        X_norm = X
    
    # Compute item-item similarity: X^T * X
    X_T = X_norm.T
    Sim = X_T.dot(X_norm)
    
    # Zero out diagonal (item similarity with itself)
    Sim.setdiag(0)
    
    # Apply minimum similarity threshold
    if min_similarity > 0:
        Sim.data[Sim.data < min_similarity] = 0
        Sim.eliminate_zeros()
    
    # Keep only top-k neighbors per item
    if k > 0:
        Sim = keep_top_k_per_row(Sim, k)
    
    print(f"Similarity computed. Shape: {Sim.shape}, Non-zeros: {Sim.nnz}, Time: {time.time() - start_time:.2f}s")
    return Sim

def keep_top_k_per_row(matrix, k):
    """Keep only top-k values per row in sparse matrix."""
    from scipy.sparse import lil_matrix
    
    lil = matrix.tolil()
    for i in range(lil.shape[0]):
        row_data = lil.data[i]
        if len(row_data) > k:
            # Get indices of top k values
            top_k_idx = np.argpartition(row_data, -k)[-k:]
            # Keep only top k
            lil.data[i] = [row_data[j] for j in top_k_idx]
            lil.rows[i] = [lil.rows[i][j] for j in top_k_idx]
    
    return lil.tocsr()

def compute_svd(X, k=100):
    """
    Compute SVD decomposition.
    
    Args:
        X: User-item interaction matrix (sparse)
        k: Number of latent factors
    
    Returns:
        U, sigma, Vt: SVD decomposition matrices
    """
    print(f"Computing SVD with k={k}...")
    start_time = time.time()
    X = X.asfptype()
    U, sigma, Vt = svds(X, k=k)
    # Reverse order (svds returns smallest singular values first)
    U, sigma, Vt = U[:, ::-1], sigma[::-1], Vt[::-1, :]
    sigma = np.diag(sigma)
    print(f"SVD computed in {time.time() - start_time:.2f}s")
    return U, sigma, Vt

def dcg_at_k(relevances, k):
    """Compute Discounted Cumulative Gain at k."""
    relevances = np.array(relevances)[:k]
    if relevances.size:
        return np.sum(relevances / np.log2(np.arange(2, relevances.size + 2)))
    return 0.0

def ndcg_at_k(recommended, relevant, k=20):
    """Compute Normalized Discounted Cumulative Gain at k."""
    rel_set = set(relevant)
    relevances = [1 if item in rel_set else 0 for item in recommended[:k]]
    dcg = dcg_at_k(relevances, k)
    ideal_relevances = [1] * min(len(rel_set), k) + [0] * (k - min(len(rel_set), k))
    idcg = dcg_at_k(ideal_relevances, k)
    return dcg / idcg if idcg > 0 else 0.0

def evaluate_hybrid_ndcg(X, Sim, U, sigma, Vt, train_hist, val_hist, alpha=0.5, k=20):
    """
    Evaluate hybrid model NDCG.
    
    Args:
        X: User-item matrix
        Sim: Item similarity matrix (ItemKNN)
        U, sigma, Vt: SVD components
        train_hist: Training history per user
        val_hist: Validation items per user
        alpha: Weight for ItemKNN (1-alpha for SVD)
        k: Top-k for NDCG calculation
    
    Returns:
        Average NDCG@k
    """
    print(f"Evaluating Hybrid NDCG@{k} (alpha={alpha})...")
    U_sigma = np.dot(U, sigma)
    ndcg_scores = []
    
    n_users = X.shape[0]
    for user in range(n_users):
        if user not in val_hist or len(val_hist[user]) == 0:
            continue
        
        # ItemKNN scores
        user_items = X[user:user+1]
        knn_scores = user_items.dot(Sim).toarray().flatten()
        
        # SVD scores
        user_vec = U_sigma[user:user+1]
        svd_scores = np.dot(user_vec, Vt).flatten()
        
        # Hybrid combination
        scores = alpha * knn_scores + (1 - alpha) * svd_scores
        
        # Mask out training items
        if user in train_hist:
            scores[train_hist[user]] = -np.inf
        
        # Get top-k recommendations
        top_k_items = np.argsort(scores)[-k:][::-1]
        ndcg = ndcg_at_k(top_k_items, val_hist[user], k)
        ndcg_scores.append(ndcg)
        
        if (user + 1) % 5000 == 0:
            print(f"  Evaluated {user + 1}/{n_users} users...")
    
    avg_ndcg = np.mean(ndcg_scores)
    print(f"NDCG@{k}: {avg_ndcg:.6f} (evaluated {len(ndcg_scores)} users)")
    return avg_ndcg

def grid_search_hybrid(X_train, train_hist, val_hist):
    """
    Perform grid search to find best hyperparameters.
    
    Returns:
        Best parameters dictionary
    """
    print("\n" + "="*70)
    print("GRID SEARCH FOR OPTIMAL HYPERPARAMETERS")
    print("="*70)
    
    # Parameter grid
    param_grid = {
        'knn_k': [20, 50, 100, 200],
        'knn_norm': ['l1', 'l2'],
        'svd_k': [50, 100, 150, 200],
        'alpha': [0.3, 0.5, 0.7]
    }
    
    print(f"\nParameter grid:")
    for param, values in param_grid.items():
        print(f"  {param}: {values}")
    
    total_combinations = np.prod([len(v) for v in param_grid.values()])
    print(f"\nTotal combinations: {total_combinations}")
    print("\nStarting grid search...\n")
    
    best_ndcg = 0.0
    best_params = {}
    results = []
    
    combo_num = 0
    for knn_k, knn_norm, svd_k, alpha in product(
        param_grid['knn_k'],
        param_grid['knn_norm'],
        param_grid['svd_k'],
        param_grid['alpha']
    ):
        combo_num += 1
        print(f"\n[{combo_num}/{total_combinations}] Testing: knn_k={knn_k}, knn_norm={knn_norm}, svd_k={svd_k}, alpha={alpha}")
        
        try:
            # Compute ItemKNN similarity
            Sim = compute_item_similarity(X_train, k=knn_k, normalization=knn_norm)
            
            # Compute SVD
            U, sigma, Vt = compute_svd(X_train, k=svd_k)
            
            # Evaluate
            ndcg = evaluate_hybrid_ndcg(X_train, Sim, U, sigma, Vt, train_hist, val_hist, alpha=alpha)
            
            results.append({
                'knn_k': knn_k,
                'knn_norm': knn_norm,
                'svd_k': svd_k,
                'alpha': alpha,
                'ndcg': ndcg
            })
            
            if ndcg > best_ndcg:
                best_ndcg = ndcg
                best_params = {
                    'knn_k': knn_k,
                    'knn_norm': knn_norm,
                    'svd_k': svd_k,
                    'alpha': alpha
                }
                print(f"  *** NEW BEST: NDCG@20 = {ndcg:.6f} ***")
            else:
                print(f"  NDCG@20 = {ndcg:.6f}")
                
        except Exception as e:
            print(f"  ERROR: {e}")
            continue
    
    print("\n" + "="*70)
    print("GRID SEARCH COMPLETE")
    print("="*70)
    print(f"\nBest NDCG@20: {best_ndcg:.6f}")
    print(f"Best parameters:")
    for param, value in best_params.items():
        print(f"  {param}: {value}")
    
    # Print top 5 results
    print("\nTop 5 configurations:")
    results_sorted = sorted(results, key=lambda x: x['ndcg'], reverse=True)[:5]
    for i, res in enumerate(results_sorted, 1):
        print(f"  {i}. NDCG={res['ndcg']:.6f}: knn_k={res['knn_k']}, knn_norm={res['knn_norm']}, svd_k={res['svd_k']}, alpha={res['alpha']}")
    
    return best_params, best_ndcg

def generate_hybrid_recommendations(X, Sim, U, sigma, Vt, output_file, alpha=0.5, top_k=20):
    """
    Generate recommendations using hybrid model.
    
    Args:
        X: User-item interaction matrix
        Sim: Item similarity matrix
        U, sigma, Vt: SVD components
        output_file: Output file path
        alpha: Weight for ItemKNN
        top_k: Number of recommendations per user
    """
    print(f"\nGenerating recommendations (alpha={alpha})...")
    n_users, batch_size = X.shape[0], 1000
    U_sigma = np.dot(U, sigma)
    
    with open(output_file, 'w') as f:
        for start_idx in range(0, n_users, batch_size):
            end_idx = min(start_idx + batch_size, n_users)
            
            # Get user batch
            user_batch = X[start_idx:end_idx]
            
            # ItemKNN scores
            knn_scores_batch = user_batch.dot(Sim).toarray()
            
            # SVD scores
            user_batch_factors = U_sigma[start_idx:end_idx]
            svd_scores_batch = np.dot(user_batch_factors, Vt)
            
            # Hybrid combination
            scores_batch = alpha * knn_scores_batch + (1 - alpha) * svd_scores_batch
            
            # Mask out training items
            mask = user_batch.toarray() > 0
            scores_batch[mask] = -np.inf
            
            # Get top-k recommendations
            top_items_indices = np.argpartition(scores_batch, -top_k, axis=1)[:, -top_k:]
            rows = np.arange(scores_batch.shape[0])[:, None]
            top_scores = scores_batch[rows, top_items_indices]
            sort_ind = np.argsort(top_scores, axis=1)[:, ::-1]
            final_recs = top_items_indices[rows, sort_ind]
            
            # Write to file
            for i, u_id in enumerate(range(start_idx, end_idx)):
                recs = final_recs[i]
                f.write(f"{u_id} {' '.join(map(str, recs))}\n")
                
            if start_idx % 5000 == 0:
                print(f"  Processed {start_idx}/{n_users} users...")
    
    print(f"Recommendations saved to: {output_file}")

def main():
    input_file = '/Users/riteshsingh/Documents/SJSU/Recommender System/projectrec/train-2.txt'
    output_file = '/Users/riteshsingh/Documents/SJSU/Recommender System/projectrec/output_hybrid.txt'
    
    # Check command line arguments
    mode = 'tune'  # Default mode
    if len(sys.argv) > 1:
        if sys.argv[1] == '--mode':
            mode = sys.argv[2] if len(sys.argv) > 2 else 'tune'
    
    print("\n" + "="*70)
    print("HYBRID RECOMMENDER SYSTEM (ItemKNN + SVD)")
    print("="*70)
    
    # Load data
    X_full, user_history = load_data(input_file)
    
    # Create train/val split
    X_train, train_hist, val_hist = create_train_val_split(user_history, X_full.shape)
    print(f"Train shape: {X_train.shape}, nnz: {X_train.nnz}")
    
    if mode == 'tune':
        # Run grid search
        best_params, best_ndcg = grid_search_hybrid(X_train, train_hist, val_hist)
        
        print("\n" + "="*70)
        print("TRAINING ON FULL DATASET WITH BEST PARAMETERS")
        print("="*70)
        
        # Train on full data with best parameters
        Sim_full = compute_item_similarity(X_full, k=best_params['knn_k'], 
                                          normalization=best_params['knn_norm'])
        U_full, sigma_full, Vt_full = compute_svd(X_full, k=best_params['svd_k'])
        
        # Generate recommendations
        generate_hybrid_recommendations(X_full, Sim_full, U_full, sigma_full, Vt_full, 
                                       output_file, alpha=best_params['alpha'])
        
        print("\n" + "="*70)
        print("FINAL RESULTS")
        print("="*70)
        print(f"Model: Hybrid (ItemKNN + SVD)")
        print(f"Validation NDCG@20: {best_ndcg:.6f}")
        print(f"Best parameters: {best_params}")
        print(f"Output file: {output_file}")
        print("="*70)
        
    elif mode == 'generate':
        # Use predefined best parameters (update these after tuning)
        best_params = {
            'knn_k': 50,
            'knn_norm': 'l2',
            'svd_k': 100,
            'alpha': 0.5
        }
        
        print(f"\nUsing parameters: {best_params}")
        
        # Train on full data
        Sim_full = compute_item_similarity(X_full, k=best_params['knn_k'], 
                                          normalization=best_params['knn_norm'])
        U_full, sigma_full, Vt_full = compute_svd(X_full, k=best_params['svd_k'])
        
        # Generate recommendations
        generate_hybrid_recommendations(X_full, Sim_full, U_full, sigma_full, Vt_full, 
                                       output_file, alpha=best_params['alpha'])
    
    print("\nDone!")

if __name__ == "__main__":
    main()



HYBRID RECOMMENDER SYSTEM (ItemKNN + SVD)
Loading data...
Data loaded. Shape: (52643, 91605), Non-zeros: 2380730
Train shape: (52643, 91605), nnz: 2328087

GRID SEARCH FOR OPTIMAL HYPERPARAMETERS

Parameter grid:
  knn_k: [20, 50, 100, 200]
  knn_norm: ['l1', 'l2']
  svd_k: [50, 100, 150, 200]
  alpha: [0.3, 0.5, 0.7]

Total combinations: 96

Starting grid search...


[1/96] Testing: knn_k=20, knn_norm=l1, svd_k=50, alpha=0.3
Computing ItemKNN similarity (k=20, norm=l1, min_sim=0.0)...
Similarity computed. Shape: (91605, 91605), Non-zeros: 1828789, Time: 50.04s
Computing SVD with k=50...
SVD computed in 2.01s
Evaluating Hybrid NDCG@20 (alpha=0.3)...
  Evaluated 5000/52643 users...
  Evaluated 10000/52643 users...
  Evaluated 15000/52643 users...
  Evaluated 20000/52643 users...
  Evaluated 25000/52643 users...
  Evaluated 30000/52643 users...
  Evaluated 35000/52643 users...
  Evaluated 40000/52643 users...
  Evaluated 45000/52643 users...
  Evaluated 50000/52643 users...
NDCG@20: 0.0